In [18]:
import pandas as pd

from dbcp.helpers import get_sql_engine

engine = get_sql_engine()

with engine.connect() as con:
    new_acp_projects = pd.read_sql_table("acp_projects", con, schema="private_data_warehouse")

In [19]:
githash = "a0fc9e0"
old_acp_projects = pd.read_parquet(f"../outputs/{githash}/dev/private_data_warehouse/acp_projects.parquet")

In [24]:
def pct_change_mw_by_status(new, old):

    new_agg = new.groupby("status").capacity_mw.sum()
    old_agg = old.groupby("status").capacity_mw.sum()
    return (new_agg - old_agg) / old_agg * 100

In [25]:
pct_change_mw_by_status(new_acp_projects, old_acp_projects)

status
Advanced Development      -11.403581
Decommissioned              2.578608
Online                      5.934032
Online | Decommissioned     0.416892
Under Construction          5.628867
Name: capacity_mw, dtype: float64

## % capacity change by status and ISO

In [30]:
new_acp_projects.iso_region.value_counts(dropna=False)

None         3588
MISO         1297
PJM          1193
CAISO        1072
ISONE         931
ERCOT         489
NYISO         418
SPP           379
WECC-RMRG       5
Name: iso_region, dtype: int64

In [31]:
for iso_region in new_acp_projects.iso_region.unique():
    new_prj_iso = new_acp_projects[new_acp_projects.iso_region == iso_region]
    old_prj_iso = old_acp_projects[old_acp_projects.iso_region == iso_region]

    pct_change = pct_change_mw_by_status(new_prj_iso, old_prj_iso)

    print(iso_region)
    print(pct_change)
    print()

CAISO
status
Advanced Development      -24.509428
Decommissioned              0.000000
Online                      4.616339
Online | Decommissioned     0.000000
Under Construction        -21.645777
Name: capacity_mw, dtype: float64

None
Series([], Name: capacity_mw, dtype: float64)

PJM
status
Advanced Development      -25.538129
Decommissioned            -22.468304
Online                      4.177580
Online | Decommissioned     0.000000
Under Construction         -3.123384
Name: capacity_mw, dtype: float64

ISONE
status
Advanced Development    0.859603
Decommissioned          0.000000
Online                  3.317459
Under Construction     -6.784373
Name: capacity_mw, dtype: float64

NYISO
status
Advanced Development   -7.567308
Online                  3.476287
Under Construction     -5.726114
Name: capacity_mw, dtype: float64

MISO
status
Advanced Development      -43.586332
Decommissioned              0.000000
Online                      3.340689
Online | Decommissioned     2.5840